<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Comparing Object and Truth Tables</b> <br>
Contact author: Jeff Carlin <br>
Last verified to run: 2022-09-23 <br>
LSST Science Piplines version: Weekly 2022_39 <br>
Container size: medium <br>
Targeted learning level: beginner <br>

**Description:** An introduction to using the truth data for the Dark Energy Science Collaboration's DC2 data set, which formed the basis for the DP0 data products.

**Skills:** Use the TAP service with table joins to retrive truth data matched to the Object catalog.

**LSST Data Products:** TAP dp02_dc2_catalogs.Object, .MatchesTruth, and .TruthSummary tables. 

**Packages:** lsst.rsp.get_tap_service, lsst.rsp.retrieve_query

**Credit:** Originally developed by Jeff Carlin and the Rubin Community Engagement Team in the context of the Rubin DP0.1.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1.0. Introduction

This tutorial demonstrates how to extract a table containing data from both the DP0.2 object and truth tables.
The query returns the table `JOIN` of these two catalogs, which enables comparison of the recovered (measured) properties (e.g., fluxes, positions, magnitudes, etc.) to the simulated values that were assigned to each object when creating the DC2 simulations.

More information about the DC2 simulations that make up DP0 can be found in [the DC2 Data Release Note](https://ui.adsabs.harvard.edu/abs/2021arXiv210104855L/abstract).

### 1.1. Package imports

The [`matplotlib`](https://matplotlib.org/), [`numpy`](http://www.numpy.org/), [`pandas`](https://pandas.pydata.org/docs/), and [`astropy`](http://www.astropy.org/) libraries are widely used Python libraries for plotting, scientific computing, and astronomical data analysis. We will use these packages below, including the `matplotlib.pyplot` plotting sublibrary.

We also use the `warnings` package to minimize standard output within the notebook, and the `lsst.rsp` package to access the TAP service and query the DP0 catalogs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
from astropy.units import UnitsWarning
import warnings
from lsst.rsp import get_tap_service, retrieve_query

### 1.2. Define functions and parameters

Ignore units warnings in this notebook, they are not cricital.

Set `matplotlib` to show plots inline, within the notebook.

Set the pandas parameter for the maximum number of rows to display to 200.

In [ ]:
warnings.simplefilter("ignore", category=UnitsWarning)
%matplotlib inline
pandas.set_option('display.max_rows', 200)

To access tables, we will use the TAP service in a similar manner to what we showed in the [Intro to DP0 notebook](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/01_Intro_to_DP0_Notebooks.ipynb), and explored further in the [TAP tutorial notebook](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/02_Intermediate_TAP_Query.ipynb). See those notebooks for more details.

In [ ]:
service = get_tap_service()

## 2.0. Discover truth data

The <a href="dp0-2.lsst.io">DP0.2 Documentation</a> contains a <a href="https://dp0-2.lsst.io/data-products-dp0-2/index.html#catalogs">list of all DP0.2 catalogs</a>, and also a link to the <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 Schema Browser</a> where users can read about the available tables and their contents.

Alternatively, the Portal Aspect of the Rubin Science Platform can be used to browse catalog data.

Below, we show how to browse catalog data from a Notebook using the TAP service.

**Optional:** Print the names of all available tables.

In [ ]:
# results = service.search("SELECT description, table_name FROM TAP_SCHEMA.tables")
# results_tab = results.to_table()

# for tablename in results_tab['table_name']:
#     print(tablename)

**Optional:** Print the table schema for the truth tables, `dp02_dc2_catalogs.MatchesTruth` and `.TruthSummary`.

Use the `.to_pandas()` method, and not just `.to_table()` (astropy table), so that all rows of the second cell display.

In [ ]:
results = service.search("SELECT column_name, datatype, description,\
                          unit from TAP_SCHEMA.columns\
                          WHERE table_name = 'dp02_dc2_catalogs.MatchesTruth'")
results.to_table().to_pandas()

In [ ]:
results = service.search("SELECT column_name, datatype, description,\
                          unit from TAP_SCHEMA.columns\
                          WHERE table_name = 'dp02_dc2_catalogs.TruthSummary'")
results.to_table().to_pandas()

# MLG EDITED UP TO HERE SO FAR

## 3. Extract a table of joined results from two tables, using a single query within ADQL

For those who are not familiar with databases, the simplest way of accomplishing the task might seem to be querying the two tables (Object and Truth-Match) separately, then matching them afterwards. This would work, but it is not the best use of the resources that are available. As seen in the [Advanced TAP/ADQL Usage in the Portal Aspect](https://dp0-1.lsst.io/tutorials-examples/index-portal-advanced.html#examples-dp0-1-portal-advanced) tutorial, the table JOIN can be done directly with ADQL. Not only will this save you a few steps, it should also execute much faster.

For this exploration, we will select a small region of sky around a random RA, Dec position. The following cell reads data centered on (RA, Dec) = (62.0, -37.0) degrees, within a radius of 0.1 degrees, from both the Object and Truth-Match table, then joins database entries where `match_objectId` from Truth-Match equals `objectId` from the Object table. Note that we are selecting only a subset of the columns seen in the schema above. You can add or remove columns as you wish.

Note that for the Object table we select all objects within the cone-shaped region of interest. In the Truth-Match table, we restrict the results to objects satisfying "match_objectId >= 0 AND is_good_match = 1". According to the Truth-Match schema above, the `is_good_match` flag is "True if this object--truth matching pair satisfies all matching criteria" as laid out in the [DESC DC2 Release Note](https://ui.adsabs.harvard.edu/abs/2021arXiv210104855L/abstract). We'll use that to select "good" matches. In the column description for `match_objectId` from above, it says "objectId of the matching object entry (-1 for unmatched truth entries)." Thus the criterion "match_objectId >= 0" removes the unmatched entries, leaving us with only the truth-table entries that were detected and appear in the Object table.


In [ ]:
# Define the query
query = "SELECT obj.objectId, obj.ra, obj.dec, obj.mag_g, obj.mag_r, "\
        "obj.mag_i, obj.mag_g_cModel, obj.mag_r_cModel, obj.mag_i_cModel, "\
        "obj.psFlux_g, obj.psFlux_r, obj.psFlux_i, obj.cModelFlux_g, "\
        "obj.cModelFlux_r, obj.cModelFlux_i, obj.tract, obj.patch, "\
        "obj.extendedness, obj.good, obj.clean, "\
        "truth.mag_r as truth_mag_r, truth.match_objectId, "\
        "truth.flux_g, truth.flux_r, truth.flux_i, truth.truth_type,  "\
        "truth.match_sep, truth.is_variable "\
        "FROM dp01_dc2_catalogs.object as obj "\
        "JOIN dp01_dc2_catalogs.truth_match as truth "\
        "ON truth.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec), "\
        "CIRCLE('ICRS', 62.0, -37.0, 0.10)) = 1 "\
        "AND truth.match_objectid >= 0 "\
        "AND truth.is_good_match = 1"
print(query)

In [ ]:
%%time
truth_plus_meas = service.search(query)
print('Query returned %s matched objects.' % len(truth_plus_meas))

Notice the `%%time` [cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) we used. This was included to highlight that selecting more than 14000 objects from two tables and joining them takes less than 2 seconds (typically). Doing two separate queries, then joining them using `pandas`, typically takes more than a full minute.

In [ ]:
# Print the names of the fields in the returned table:

truth_plus_meas.fieldnames

## 4. Compare table values by plotting

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 18,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [10, 8],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

#### Compare the measurements from the Object table to the "true" values for some objects.

To do this, we will separate the "stars" and "galaxies" using the `truth_type` column from the Truth-Match table. Simulated stars have `truth_type = 2`, and galaxies, `truth_type = 1`.

After separating stars and galaxies, we'll compare the recovered flux to the "true" value that was simulated for each object (as a ratio of the fluxes).

In [ ]:
star = np.where(truth_plus_meas['truth_type'] == 2)
gx = np.where(truth_plus_meas['truth_type'] == 1)

Just to confirm that things look like we expect, let's plot a color-magnitude (g vs. g-i) and color-color (r-i vs. g-r) diagram.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 8))

plt.sca(ax[0])  # set the first axis as current

plt.plot(truth_plus_meas['mag_g_cModel'][gx] - truth_plus_meas['mag_i_cModel'][gx],
         truth_plus_meas['mag_g_cModel'][gx], 'k.', alpha=0.2, label='galaxies')
plt.plot(truth_plus_meas['mag_g_cModel'][star] - truth_plus_meas['mag_i_cModel'][star],
         truth_plus_meas['mag_g_cModel'][star], 'ro', label='stars')
plt.legend(loc='upper left')
plt.xlabel(r'$(g-i)$')
plt.ylabel(r'$g$')
plt.xlim(-1.8, 4.3)
plt.ylim(29.3, 16.7)
plt.minorticks_on()

plt.sca(ax[1])  # set the first axis as current
plt.plot(truth_plus_meas['mag_g_cModel'][gx] - truth_plus_meas['mag_r_cModel'][gx],
         truth_plus_meas['mag_r_cModel'][gx] - truth_plus_meas['mag_i_cModel'][gx],
         'k.', alpha=0.1, label='galaxies')
plt.plot(truth_plus_meas['mag_g_cModel'][star] - truth_plus_meas['mag_r_cModel'][star],
         truth_plus_meas['mag_r_cModel'][star] - truth_plus_meas['mag_i_cModel'][star],
         'ro', label='stars')
plt.legend(loc='upper left')
plt.xlabel(r'$(g-r)$')
plt.ylabel(r'$(r-i)$')
plt.xlim(-1.3, 2.3)
plt.ylim(-1.3, 2.8)
plt.minorticks_on()

plt.tight_layout()
plt.show()

Looks pretty normal - the stellar locus in color-color space is right where one expects it to be, and the galaxies dominate at the faint end of the CMD. 

Now let's compare the fluxes:

In [ ]:
plt.rcParams.update({'figure.figsize': (11, 10)})

plt.plot(truth_plus_meas['truth_mag_r'][gx],
         truth_plus_meas['cModelFlux_r'][gx] / truth_plus_meas['flux_r'][gx],
         'k.', alpha=0.2, label='galaxies')
plt.plot(truth_plus_meas['truth_mag_r'][star],
         truth_plus_meas['cModelFlux_r'][star] / truth_plus_meas['flux_r'][star],
         'ro', label='stars')
plt.legend(loc='upper left')
plt.xlabel(r'$r$ magnitude (truth)')
plt.ylabel(r'$f_{\rm meas}/f_{\rm truth}$')
plt.ylim(0.15, 2.15)
plt.xlim(17.6, 27.8)
plt.minorticks_on()
plt.show()

Well, that looks good -- the ratio of measured to true fluxes is centered on 1.0. It seems like the fluxes are recovered pretty well, on average.

Congratulations! You have now learned how to compare measurements in the DP0.1 catalogs to the "true" simulated properties of objects. Have fun exploring more properties!